# CHAPTER6: Odds And Addends

## Libraries

In [ ]:
!pip install empiricaldist statsmodels matplotlib

In [16]:
# Get utils.py
from os.path import basename, exists

# def download(url):
#     filename = basename(url)
#     if not exists(filename):
#         from urllib.request import urlretrieve
#         local, _ = urlretrieve(url, filename)
#         print('Downloaded ' + local)
    
# download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

from utils import decorate
from utils import set_pyplot_params
set_pyplot_params()

import numpy as np
from empiricaldist import Pmf
import matplotlib.pyplot as plt
from utils import make_binomial
import pandas as pd


In [10]:
def odds(p):
    return p / (1-p)
def prob(o):
    return o / (o+1)
def prob2(yes, no):
    return yes / (yes + no)
def add_dist(pmf1, pmf2):
    """Compute the distribution of a sum."""
    res = Pmf()
    for q1, p1 in pmf1.items():
        for q2, p2 in pmf2.items():
            q = q1 + q2
            p = p1 * p2
            res[q] = res(q) + p
    return res

In [8]:
def make_die(sides):
    outcomes = np.arange(1, sides+1)
    die = Pmf(1/sides, outcomes)
    return die

In [13]:
def add_dist(pmf1, pmf2):
    """Compute the distribution of a sum."""
    res = Pmf()
    for q1, p1 in pmf1.items():
        for q2, p2 in pmf2.items():
            q = q1 + q2
            p = p1 * p2
            res[q] = res(q) + p
    return res
def add_dist_seq(seq):
    """Compute Pmf of the sum of values from seq."""
    total = seq[0]
    for other in seq[1:]:
        total = total.add_dist(other)
    return total

In [12]:
from utils import decorate

def decorate_dice(title=''):
    decorate(xlabel='Outcome',
             ylabel='PMF',
             title=title)

## Exercises

  - Oliver Exercise
    - [Graph]("https://www.desmos.com/calculator/zija07zc68")

In [22]:
# a / (1+a) vs al / (1 + al)
a = np.array([0.01, 1, 0.99])
l = 0.83
a / (1+a), a*l / (1 + a*l)

(array([0.00990099, 0.5       , 0.49748744]),
 array([0.00823168, 0.45355191, 0.45106219]))

- Elvis Presley

In [ ]:
# Solution
prior_odds = odds(1/3)
post_odds = prior_odds * 2 * 1.25
prob(post_odds)

In [19]:
prob(0.83), prob(9*0.83), prob(1/9 * 0.83)

(0.45355191256830596, 0.8819362455726093, 0.0844354018311292)

In [20]:
1/9

0.1111111111111111